In [1]:
# Import Statements
import os

import pandas as pd
import numpy as np

# Load Dataset files
    We will be working with the following files:
    * Training set values
    * Training set labels
    * Test set values


In [2]:
train_values = pd.read_csv("./datasets/train_values.csv")
train_labels = pd.read_csv("./datasets/train_labels.csv")

test_values = pd.read_csv("./datasets/test_values.csv")

In [15]:
num_class = 3
classes = ("functional", "functional needs repair","non functional")

## Show Train Values

In [4]:
print(train_values)
print("*" * 89)
print()

id  amount_tsh date_recorded           funder  gps_height  \
0      69572      6000.0    2011-03-14            Roman        1390   
1       8776         0.0    2013-03-06          Grumeti        1399   
2      34310        25.0    2013-02-25     Lottery Club         686   
3      67743         0.0    2013-01-28           Unicef         263   
4      19728         0.0    2011-07-13      Action In A           0   
...      ...         ...           ...              ...         ...   
59395  60739        10.0    2013-05-03  Germany Republi        1210   
59396  27263      4700.0    2011-05-07      Cefa-njombe        1212   
59397  37057         0.0    2011-04-11              NaN           0   
59398  31282         0.0    2011-03-08            Malec           0   
59399  26348         0.0    2011-03-23       World Bank         191   

          installer  longitude   latitude              wpt_name  num_private  \
0             Roman  34.938093  -9.856322                  none            0 

## Show Train Labels

In [5]:
print(train_labels)
print("*" * 89)

id    status_group
0      69572      functional
1       8776      functional
2      34310      functional
3      67743  non functional
4      19728      functional
...      ...             ...
59395  60739      functional
59396  27263      functional
59397  37057      functional
59398  31282      functional
59399  26348      functional

[59400 rows x 2 columns]
*****************************************************************************************


## Show Test Values

In [6]:
print(test_values)
print("*" * 89)

id  amount_tsh date_recorded                  funder  gps_height  \
0      50785         0.0    2013-02-04                    Dmdd        1996   
1      51630         0.0    2013-02-04  Government Of Tanzania        1569   
2      17168         0.0    2013-02-01                     NaN        1567   
3      45559         0.0    2013-01-22              Finn Water         267   
4      49871       500.0    2013-03-27                  Bruder        1260   
...      ...         ...           ...                     ...         ...   
14845  39307         0.0    2011-02-24                  Danida          34   
14846  18990      1000.0    2011-03-21                    Hiap           0   
14847  28749         0.0    2013-03-04                     NaN        1476   
14848  33492         0.0    2013-02-18                 Germany         998   
14849  68707         0.0    2013-02-13  Government Of Tanzania         481   

        installer  longitude   latitude                 wpt_name  num_pri

In [7]:

# Count the number of missing values per column
#print(train_values.describe())
#print("-" * 89)
print(train_values.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [8]:
# Count the number of missing values per column
#print(test_values.describe())
#print("-" * 89)
print(test_values.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     14850 non-null  int64  
 1   amount_tsh             14850 non-null  float64
 2   date_recorded          14850 non-null  object 
 3   funder                 13981 non-null  object 
 4   gps_height             14850 non-null  int64  
 5   installer              13973 non-null  object 
 6   longitude              14850 non-null  float64
 7   latitude               14850 non-null  float64
 8   wpt_name               14850 non-null  object 
 9   num_private            14850 non-null  int64  
 10  basin                  14850 non-null  object 
 11  subvillage             14751 non-null  object 
 12  region                 14850 non-null  object 
 13  region_code            14850 non-null  int64  
 14  district_code          14850 non-null  int64  
 15  lg

In [9]:
column_labels = list(train_values.columns.values)
column_labels.remove("id")
column_labels.remove("amount_tsh")
column_labels.remove("date_recorded")
column_labels.remove("gps_height")
column_labels.remove("longitude")
column_labels.remove("latitude")
column_labels.remove("num_private")
column_labels.remove("region_code")
column_labels.remove("district_code")
column_labels.remove("population")
column_labels.remove("construction_year")

In [10]:
test_values = test_values.fillna(test_values.median())

In [11]:
for col in column_labels:
    unique_values = list(set(np.concatenate((train_values[col].unique(), test_values[col].unique()))))
    size = len(unique_values)
    print(size)
    for s in range(size):
        if unique_values[s] != "nan":
            train_values.loc[train_values[col] == unique_values[s], col] = s
            test_values.loc[test_values[col] == unique_values[s], col] = s

train_values = train_values.fillna(train_values.median())
test_values = test_values.fillna(test_values.median())

2141
2411
45684
9
21426
21
125
2098
3
1
13
2869
3
18
13
7
12
5
7
7
8
6
5
5
10
7
3
7
6


In [12]:
train_values.to_csv("./datasets/preprocessed_data/train.csv", index=False, header=True)
test_values.to_csv("./datasets/preprocessed_data/test.csv", index=False, header=True)

In [16]:
train_labels_int = train_labels.copy()
for idx, data in train_labels_int["status_group"].iteritems():
    if data in classes:
        train_labels_int["status_group"][idx] = classes.index(data)

In [17]:
train_labels_int.head(10)

,id,status_group
0,69572,0
1,8776,0
2,34310,0
3,67743,2
4,19728,0
5,9944,0
6,19816,2
7,54551,2
8,53934,2
9,46144,0


In [18]:
train_labels_int.to_csv("./datasets/preprocessed_data/train_labels.csv", index=False, header=True)

In [19]:
if train_values["id"].equals(train_labels_int["id"]):
    print("True")
    train_values["status_group"] = train_labels_int["status_group"]

True


In [20]:
train_values.head(5)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,1559.0,1390,1764.0,34.938093,-9.856322,5520,0,...,2,5,3,3,3,2,2,0,0,0
1,8776,0.0,2013-03-06,1731.0,1399,1868.0,34.698766,-2.147466,17493,0,...,2,5,4,4,0,0,0,0,0,0
2,34310,25.0,2013-02-25,1431.0,686,1732.0,37.460664,-3.821329,37342,0,...,2,5,3,3,5,3,0,3,0,0
3,67743,0.0,2013-01-28,1265.0,263,1254.0,38.486161,-11.155298,12,0,...,2,5,1,1,4,5,2,3,0,2
4,19728,0.0,2011-07-13,1555.0,0,1454.0,31.130847,-1.825359,29877,0,...,2,5,2,2,0,0,0,0,0,0


In [21]:
train_values.to_csv("./datasets/preprocessed_data/merged_train.csv", index=False, header=True)